In [69]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from tensorflow.keras.callbacks import TensorBoard

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 2.2.0
Eager execution: True


In [70]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
test_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)
test_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(test_dataset_url),
                                          origin=test_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))

Local copy of the dataset file: C:\Users\LONAA32\.keras\datasets\iris_training.csv


In [71]:
def create_dataset(file_path, batch_size=1):

    dataset = tf.data.experimental.make_csv_dataset(file_path,
                                                    batch_size,
                                                    column_names=COLUMN_NAMES,
                                                    label_name=LABEL_NAME)
    return dataset

In [72]:
# column order in CSV file
COLUMN_NAMES = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
FEATURE_NAMES = COLUMN_NAMES[:-1]
LABEL_NAME = COLUMN_NAMES[-1]

print("Features: {}".format(FEATURE_NAMES))
print("Label: {}".format(LABEL_NAME))

Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
Label: species


In [89]:
BATCH_SIZE = 32

train_dataset = create_dataset(train_dataset_fp, batch_size=BATCH_SIZE)
test_dataset = create_dataset(test_dataset_fp, batch_size=BATCH_SIZE)

In [ ]:
features, labels = next(iter(train_dataset))

print(features)

In [6]:
for element in train_dataset.as_numpy_iterator():
    print(element)
    break

(OrderedDict([('sepal_length', array([7.2], dtype=float32)), ('sepal_width', array([3.], dtype=float32)), ('petal_length', array([5.8], dtype=float32)), ('petal_width', array([1.6], dtype=float32))]), array([2]))


In [23]:
def pack_features_vector(features, labels):
    """Pack the features into a single array."""
    features = tf.stack(list(features.values()), axis=1)
    return features, labels

In [24]:
train_dataset = train_dataset.map(pack_features_vector)

In [26]:
features, labels = next(iter(train_dataset))

print(features)
print(labels)

tf.Tensor(
[[5.2 2.7 3.9 1.4]
 [6.8 3.2 5.9 2.3]
 [6.  2.9 4.5 1.5]
 [5.5 2.4 3.8 1.1]
 [5.9 3.  5.1 1.8]
 [5.8 2.7 4.1 1. ]
 [5.7 4.4 1.5 0.4]
 [6.9 3.1 4.9 1.5]
 [4.9 3.1 1.5 0.1]
 [4.9 3.1 1.5 0.1]
 [6.5 3.2 5.1 2. ]
 [7.7 3.  6.1 2.3]
 [6.7 3.  5.  1.7]
 [7.7 2.6 6.9 2.3]
 [5.6 2.5 3.9 1.1]
 [4.9 3.  1.4 0.2]
 [6.4 2.7 5.3 1.9]
 [5.7 2.8 4.5 1.3]
 [5.4 3.  4.5 1.5]
 [6.1 2.8 4.7 1.2]
 [5.  3.5 1.6 0.6]
 [6.7 3.  5.2 2.3]
 [6.5 3.  5.8 2.2]
 [6.8 2.8 4.8 1.4]
 [4.7 3.2 1.6 0.2]
 [5.7 3.8 1.7 0.3]
 [6.4 2.8 5.6 2.1]
 [4.4 3.  1.3 0.2]
 [6.3 2.5 5.  1.9]
 [5.9 3.2 4.8 1.8]
 [6.3 2.3 4.4 1.3]
 [5.5 2.4 3.7 1. ]], shape=(32, 4), dtype=float32)
tf.Tensor([1 2 1 1 2 1 0 1 0 0 2 2 1 2 1 0 2 1 1 1 0 2 2 1 0 0 2 0 2 1 1 1], shape=(32,), dtype=int32)


In [74]:
# Create input layer of feature columns
feature_columns = {header: tf.feature_column.numeric_column(header) for header in FEATURE_NAMES}

In [75]:
# Build a keras DNN model using Sequential API
# TODO 2a
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3, activation="softmax")  
])

In [82]:
# Build a keras DNN model using Sequential API
# TODO 2a
model = tf.keras.Sequential([
  tf.keras.layers.DenseFeatures(feature_columns=feature_columns.values()),
  tf.keras.layers.Dense(10, activation=tf.nn.relu),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3, activation="softmax")  
])

In [76]:
predictions = model(features)
predictions[:5]

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0.422602  , 0.06274351, 0.5146545 ],
       [0.3391083 , 0.02939623, 0.6314955 ],
       [0.40758455, 0.04595266, 0.5464628 ],
       [0.40852743, 0.05661747, 0.53485507],
       [0.38273373, 0.04433028, 0.572936  ]], dtype=float32)>

In [83]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [84]:
# Compile the keras model
model.compile(optimizer="adam",
             loss=loss_object,
             metrics=["mse", "accuracy"])

In [85]:
history = model.fit(train_dataset,
                    steps_per_epoch=120,
                    epochs=50)

Epoch 1/50
120/120 [==============================] - 0s 950us/step - loss: 0.9969 - mse: 1.1738 - accuracy: 0.5516
Epoch 2/50
120/120 [==============================] - 0s 867us/step - loss: 0.8056 - mse: 1.2593 - accuracy: 0.7005
Epoch 3/50
120/120 [==============================] - 0s 831us/step - loss: 0.7536 - mse: 1.2712 - accuracy: 0.7883
Epoch 4/50
120/120 [==============================] - 0s 875us/step - loss: 0.6893 - mse: 1.2858 - accuracy: 0.9526
Epoch 5/50
120/120 [==============================] - 0s 863us/step - loss: 0.6344 - mse: 1.3129 - accuracy: 0.9758
Epoch 6/50
120/120 [==============================] - 0s 883us/step - loss: 0.6099 - mse: 1.3295 - accuracy: 0.9766
Epoch 7/50
120/120 [==============================] - 0s 933us/step - loss: 0.5982 - mse: 1.3383 - accuracy: 0.9784
Epoch 8/50
120/120 [==============================] - 0s 899us/step - loss: 0.5918 - mse: 1.3435 - accuracy: 0.9786
Epoch 9/50
120/120 [==============================] - 0s 858us/step - lo

In [86]:
test_dataset = test_dataset.map(pack_features_vector)

In [87]:
test_dataset

<MapDataset shapes: ((32, 4), (32,)), types: (tf.float32, tf.int32)>

In [90]:
test_accuracy = tf.keras.metrics.Accuracy()

for (x, y) in test_dataset.take(50):
    # training=False is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    logits = model(x, training=False)
    prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
    test_accuracy(prediction, y)

    
print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

Test set accuracy: 93.313%


In [91]:
tf.stack([y,prediction],axis=1)

<tf.Tensor: shape=(32, 2), dtype=int32, numpy=
array([[1, 1],
       [1, 1],
       [0, 0],
       [2, 2],
       [1, 1],
       [1, 2],
       [0, 0],
       [1, 1],
       [1, 1],
       [2, 2],
       [2, 2],
       [0, 0],
       [1, 1],
       [2, 2],
       [2, 2],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [2, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [2, 2],
       [1, 1],
       [1, 2],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])>